In [ ]:
"""
!git clone https://github.com/ultralytics/yolov5
%pip install pycocotools -qr yolov5/requirements.txt  # install dependencies
!pip install GPUtil
"""

In [1]:
!wandb login 095094adaf331ede4246560053f1a391b197c03c

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
config_file = "config/original_1024_x.json"

IS_TEST = True

In [3]:
import json
import os
import datetime
import yaml
from tqdm.notebook import tqdm
from glob import glob
import numpy as np 
import pandas as pd

CFG = json.load(open(config_file))
config_filename = os.path.splitext(os.path.basename(config_file))[0]

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()
free_gpu_cache()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  2% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  0% |  3% |


In [4]:
os.chdir('yolov5')

In [ ]:
for fold in range(CFG["fold_num"]):
    if fold==1 and IS_TEST==True:
        break
    exp_name = f'{config_filename}_{fold}'
    hyp_conf_file = f'../chest_yolo/{CFG["hyp_conf"]}.yaml'
    data_file = f"../chest_yolo/{config_filename}/yolo_{fold}.yaml"

    epochs = CFG["epochs"]
    bach_size = CFG["bach_size"]
    weights  = CFG["weights"]
    dim = CFG["dim"]
    
    free_gpu_cache()
        
    !python train.py \
     --epochs $epochs \
     --batch-size $bach_size \
     --hyp $hyp_conf_file \
     --data $data_file \
     --weights $weights \
     --img $dim \
     --name $exp_name
      

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  1% |  3% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  1% |  3% |
github: up to date with https://github.com/ultralytics/yolov5 ✅
2021-03-20 23:58:36.977121: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.2
wandb: Currently logged in as: kami (use `wandb login --relogin` to force relogin)
2021-03-20 23:58:42.148960: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.2
2021-03-20 23:58:42.151684: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.2
wandb: Tracking run with wandb version 0.10.22
wandb: Syncing run original_1024_x_0
wandb: ⭐️ View project at https://wandb.ai/kami/YOLOv5
wandb: 🚀 View run at https://wandb.ai/kami/YOLOv5/runs/53zr946e
wandb: Run data is saved locally in /home/kag

In [ ]:
os.chdir('..')

In [ ]:
for fold in range(CFG["fold_num"]):
    if fold==1 and IS_TEST==True:
        break
    exp_name = f'{config_filename}_{fold}'
    weights = f'runs/train/{exp_name}/weights/best.pt'
    TEST_PATH = f'../../input/vinbigdata-chest-xray-resized-png-{CFG["dim"]}x{CFG["dim"]}{CFG["way"]}/test'

    !python detect.py --weights $weights \
    --img-size 512 --conf-thres 0.005 \
    --source $TEST_PATH --iou-thres 0.45 \
    --save-txt --save-conf \
    --name $exp_name

In [ ]:
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

In [ ]:
for fold in range(CFG["fold_num"]):
    if fold==1 and IS_TEST==True:
        break
    test_df = pd.read_csv(f'../input/test.csv')
    exp_name = f'{config_filename}_{fold}'
    detect_path = f'yolov5/runs/detect/{exp_name}/labels/*txt'
    

    image_ids = []
    PredictionStrings = []

    for file_path in tqdm(glob(detect_path)):
        image_id = file_path.split('/')[-1].split('.')[0]
        w, h = test_df.loc[test_df.image_id==image_id,['width', 'height']].values[0]
        f = open(file_path, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 1).astype(str))
        for idx in range(len(bboxes)):
            bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
        image_ids.append(image_id)
        PredictionStrings.append(' '.join(bboxes))

    pred_df = pd.DataFrame({'image_id':image_ids,
                            'PredictionString':PredictionStrings})
    sub_df = pd.merge(test_df, pred_df, on = 'image_id', how = 'left').fillna("14 1 0 0 1 1")
    sub_df = sub_df[['image_id', 'PredictionString']]
    sub_df.to_csv(f'output/{exp_name}_sub.csv',index = False)